### Test Instructions
The purpose of this simple coding test is to allow us to get a grasp on the quality of your code and to identify the role that would suit you best within our team. You don't NEED to be able to answer all of the items, but the ones that you do answer need to be correct. Feel free to use Google or any other tools that you prefer to complete these tasks.

Install any needed third-party libraries below this block. Please install the minimum amount of libraries you need.

!pip install whatever_you_need

### Task 1 (Data manipulation / Visualization)


In `message_data.csv` there's two columns, one corresponds to the unique id of someone sending a message and the other one to when they sent it.

Generate an interactive chart that shows the percentage of daily messages sent by new users (those who sent their first message ever on a given day).

In [ ]:
import pandas as pd
import plotly.express as pex
from datetime import datetime




csv = pd.read_csv('https://drive.google.com/u/0/uc?id=1R_M3xI3b_BqAN_xWLqJDneg273X_sok5&export=download') # might fail, reinclude data

#print(csv)
prev_time = csv['message_time'][0].split()[0]

print(prev_time)
data = {'day':[], 'percentage_new': []}
total = set()
daily = set()
t_cnt = 0
d_cnt = 0
d_unique = 0
for r, (mid, aid, m_time) in csv.iterrows():
    m_time = m_time.split()[0]
    if m_time != prev_time:
        date = datetime.strptime(prev_time, '%Y-%m-%d')
        #print(len(seen), d_cnt, date)
        data['day'].append(date)
        data['percentage_new'].append(len(daily)/d_cnt * 100)
        prev_time = m_time
        d_cnt = 0
        daily = set()

    daily.add(aid)
    total.update(daily)
    d_cnt += 1


data['day'].append(datetime.strptime(m_time, '%Y-%m-%d'))
data['percentage_new'].append(len(daily) / d_cnt * 100)

chart = pex.line(data, x = 'day', y='percentage_new', title='Percentage of daily messages sent by new users')
chart.show()





2022-05-01


### Task 2 (Django)
**Django follow certain instructions.**

In [ ]:
# Submitted as video

### Task 3 (LangChain)
Write a simple Chain that does the following:
- Uses openai's gpt-3.5-turbo model.
- Remembers the last 2 message exchanges, as well as the system message.
- Greets people and tells a joke about their name.

In [ ]:
import openai
import os
from collections import deque

"""
Uses openai's gpt-3.5-turbo model.
Remembers the last 2 message exchanges, as well as the system message.
Greets people and tells a joke about their name.
"""


def get_token():
    dirs = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
    with open(dirs + '\gpt.txt', 'r+') as file:
        token = file.read()
    token = token.split('\n')
    return token[0]


if __name__ == '__main__':
    c = {'system': deque(), 'message': deque()}
    # example message
    user_name = 'xxxx'
    c['system'].append(
        f'You are a personal assistant of {user_name} who is currently located in Chicago. You greet him with a joke about his name')
    c['message'].appendleft('Hello, tell me a good chinese restaurant in my location')
    c['message'].appendleft('Hello, how is the weather today')
    #
    openai.api_key = get_token()
    res = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[{'role': 'system',
                   'content': ''.join(c['system'])},
                  {'role': 'user',
                   'content': '\n'.join(c['message'])}
                  ],
        temperature=0.075
    )
    # message retaining
    # while len(c['message']) > 2:
    #     c['message'].pop()
    print(res)
    #print(c['choices'][0]['message']['content'])

"""
Hello xxxx! How's the Windy City treating you today? Speaking of wind, did you know that your name "Zhang" sounds like "jang" which is the sound the wind makes? Just a little joke to brighten up your day! Now, let's find you a delicious Chinese restaurant nearby.
"""

### Task 4 (Discord API)

Write a very simple discord bot that says "Hello {user_name}!" to every new incoming message in a discord server.


In [ ]:
import os
import discord

def get_token():
    dirs = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
    with open(dirs + '\discord.txt', 'r+') as file:
        token = file.read()
    token = token.split('\n')
    return token[1]

class DiscordClient(discord.Client):
    async def on_ready(self):
        print(f'logged in as {self.user}')

    async def on_message(self, message):
        if message.content.startswith('Hey'):
            await message.channel.send('Hello')
        print(f'Message from {message.author}: {message.content}')


    async def on_member_join(self, member):
        #print('someone joined', client, client.guilds)
        await member.guild.text_channels[0].send(f'Hello {member.name}!')



if __name__ == '__main__':
    intents = discord.Intents.all()
    intents.message_content = True
    client = DiscordClient(intents=intents)
    client.run(get_token())


### Task 5
Use LangChain to write a discord bot that will:
- Greet a user, making a joke about their name and then proceed to answer any question they have, whenever a user sends a message.
- Use async with langchain chain calls.


In [ ]:
import openai
import os
import discord


def get_token():
    dirs = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
    token = []
    with open(dirs + '\gpt.txt', 'r+') as file:
        token.append(file.read().split('\n')[0])
    with open(dirs + '\discord.txt', 'r+') as file:
        token.append(file.read().split('\n')[1])
    return token


class DiscordClient(discord.Client):

    async def on_ready(self):
        print(f'logged in as {self.user}')
        openai.api_key = get_token()[0]

    async def on_message(self, message):
        if message.author == self.user:
            return
        print(f'Message from {message.author}: {message.content}')
        res = await openai.ChatCompletion.acreate(
            model='gpt-3.5-turbo',
            messages=[{'role': 'system', 'content': "You're a discord manager and also a wikipedia"},
                      {'role': 'user', 'content': message.content}],
            temperature=0,
            max_tokens=128,
        )
        #print(res)
        content = res['choices'][0]['message']['content']
        await message.channel.send(f'{message.author.mention}, {content}')

    async def on_member_join(self, member):
        # print('someone joined', client, client.guilds)
        res = await openai.ChatCompletion.acreate(
            model='gpt-3.5-turbo',
            messages=[{'role': 'system',
                       'content': 'You are a discord doorman'},
                      {'role': 'user', 'content': f'make a joke about username {member.name}'}],
            temperature=0.075,
        )
        #print(res)
        content = res['choices'][0]['message']['content']
        await member.guild.text_channels[0].send(f'Hello {member.mention}! \n {content}')


if __name__ == '__main__':
    intents = discord.Intents.all()
    client = DiscordClient(intents=intents)
    client.run(get_token()[1])

"""
p: new user lua24
a:  Why did the Lua24 username get kicked out of the Discord party?

Because they were too good at scripting their way into trouble!

--

q: who wrote 1984
a: The novel "1984" was written by George Orwell. It was published in 1949 and has since become a classic dystopian novel, exploring themes of totalitarianism, government surveillance, and the manipulation of truth.
"""
